<a href="https://colab.research.google.com/github/Rocking-Priya/704-fall-projects-2025/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DX 704 Week 12 Project
This week's project will revisit the email spam classifier project from week 9 using large language model embeddings instead of custom features.


The full project description and a template notebook are available on GitHub: [Project 12 Materials](https://github.com/bu-cds-dx704/dx704-project-12).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Download Data Set

We will be using the Enron spam data set as prepared in this GitHub repository.

https://github.com/MWiechmann/enron_spam_data

You may need to download this differently depending on your environment.

In [3]:
!wget https://github.com/MWiechmann/enron_spam_data/raw/refs/heads/master/enron_spam_data.zip

--2025-11-20 04:40:49--  https://github.com/MWiechmann/enron_spam_data/raw/refs/heads/master/enron_spam_data.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MWiechmann/enron_spam_data/refs/heads/master/enron_spam_data.zip [following]
--2025-11-20 04:40:49--  https://raw.githubusercontent.com/MWiechmann/enron_spam_data/refs/heads/master/enron_spam_data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15642124 (15M) [application/zip]
Saving to: ‘enron_spam_data.zip’

enron_spam_data.zip 100%[===================>]  14.92M  --.-KB/s    in 0.04s   

2025-11-20 04:40:50 (387 MB/s) - ‘enron_

In [4]:
import pandas as pd

In [5]:
# pandas can read the zip file directly
enron_spam_data = pd.read_csv("enron_spam_data.zip")
enron_spam_data

,Message ID,Subject,Message,Spam/Ham,Date
0,0,christmas tree farm pictures,NaN,ham,1999-12-10
1,1,"vastar resources , inc .","gary , production from the high island larger ...",ham,1999-12-13
2,2,calpine daily gas nomination,- calpine daily gas nomination 1 . doc,ham,1999-12-14
3,3,re : issue,fyi - see note below - already done .\nstella\...,ham,1999-12-14
4,4,meter 7268 nov allocation,fyi .\n- - - - - - - - - - - - - - - - - - - -...,ham,1999-12-14
...,...,...,...,...,...
33711,33711,= ? iso - 8859 - 1 ? q ? good _ news _ c = eda...,"hello , welcome to gigapharm onlinne shop .\np...",spam,2005-07-29
33712,33712,all prescript medicines are on special . to be...,i got it earlier than expected and it was wrap...,spam,2005-07-29
33713,33713,the next generation online pharmacy .,are you ready to rock on ? let the man in you ...,spam,2005-07-30
33714,33714,bloow in 5 - 10 times the time,learn how to last 5 - 10 times longer in\nbed ...,spam,2005-07-30


In [6]:
(enron_spam_data["Spam/Ham"] == "spam").mean()

np.float64(0.5092834262664611)

## Part 2: Download BERT Model

We will use a pre-trained BERT model to extract embedding vectors as described in lesson 2.1 this week.
Here is sample code to download the model from [Hugging Face](https://huggingface.co/) and extract one vector.
This model is small enough that you can run it with CPU only, but GPUs will be faster if available.

In [7]:
# You may need to install torch and transformers.
# Google Colab will have these installed already.
#
# pip install transformers torch --upgrade

import torch
from transformers import AutoTokenizer, AutoModel

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

To download the pre-trained model from Hugging Face, you will need to sign up for a free account with them at https://huggingface.co/join.
Afterwards, get an API token and if you are using Google Colab, save it as a secret named HF_TOKEN.

In [9]:
MODEL_NAME = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
bert_model = AutoModel.from_pretrained(MODEL_NAME)
bert_model.to(device)
bert_model.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [10]:
@torch.no_grad()
def embed_text(text):
    batch = [text]
    inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt").to(device)
    outputs = bert_model(**inputs)
    # CLS token embedding is the first token's hidden state
    cls_emb = outputs.last_hidden_state[:, 0, :]  # shape: [batch_size, 768]
    return cls_emb.cpu()

In [11]:
v = embed_text("Hi, will you buy my spam?")
v.shape

torch.Size([1, 768])

## Part 3: Create Embedding Vectors

Use BERT to create embeddings for each email in the Enron data set.
You will have to decide how to combine the different columns of the original data set to produce one embedding vector.


Hint: BERT can be run without a GPU, but will be much slower.
Using Google Colab with only a CPU, it runs around 1 embedding per second.
Using Google Colab with the T4 GPU option, it runs around 60 embeddings per second.
Caching is also encouraged to avoid unnecessary reruns.

In [12]:
# YOUR CHANGES HERE

import json
import numpy as np
import pandas as pd
from tqdm import tqdm

# 1. Decide how to combine text columns into one input string
# The dataset normally has 'Subject' and 'Message' (sometimes different names)
def combine_text(row):
    # safest: join all text-like columns except label + ID
    text_parts = []
    for col in row.index:
        if col.lower() in ["spam/ham", "message id", "spam", "ham"]:
            continue
        value = row[col]
        if isinstance(value, str):
            text_parts.append(value)
    return "\n".join(text_parts)


# 2. Create embeddings for every email
records = []

for row in tqdm(enron_spam_data.itertuples(), total=len(enron_spam_data)):
    msg_id = row._asdict().get("Message ID", row.Index)   # fallback if column name differs

    combined_text = combine_text(enron_spam_data.loc[msg_id])

    emb = embed_text(combined_text)[0].numpy()    # shape (768,)
    emb_list = [round(float(x), 3) for x in emb]                       # convert to 1-D python list

    emb_json = json.dumps(emb_list)               # convert list → json string

    records.append((msg_id, emb_json))





100%|██████████| 33716/33716 [10:18<00:00, 54.48it/s]


Save your embeddings in a file "embeddings.tsv.gz" with two columns, Message ID and embedding_vector_json, where embedding_vector_json is a JSON-encoded list.
Make sure that embedding_vector_json is a 1 dimensional list, not 2 dimensional.

Hint: don't forget the ".gz" extension indicating gzip compression.
The Pandas `.to_csv` method will automatically add the compression if you save data with a filename ending in ".gz", so you just need to pass it the right filename.

In [14]:
# YOUR CHANGES HERE

# 3. Save to DataFrame
df_emb = pd.DataFrame(records, columns=["Message ID", "embedding_vector_json"])

# 4. Save to embeddings.tsv.gz
df_emb.to_csv("embeddings.tsv.gz", sep="\t", index=False)


In [15]:
import google.colab
google.colab.files.download("embeddings.tsv.gz")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
print(df_emb.shape)
df_emb.head()

(33716, 2)


,Message ID,embedding_vector_json
0,0,"[-0.318, -0.183, -0.222, -0.516, -0.126, -0.35..."
1,1,"[-0.606, -0.05, 0.192, 0.113, 0.343, -0.239, -..."
2,2,"[-0.911, -0.221, -0.179, -0.279, 0.12, -0.345,..."
3,3,"[-0.721, 0.054, -0.164, 0.117, -0.331, -0.205,..."
4,4,"[-0.693, -0.155, 0.405, 0.062, -0.195, 0.035, ..."


In [17]:
import pandas as pd
import json

# Load the gzipped TSV
df = pd.read_csv("embeddings.tsv.gz", sep="\t")

# Check columns
print(df.columns)

# Check first row
print(df.iloc[0])

# Check that embedding is 1-dimensional list
first_vector = json.loads(df.iloc[0]['embedding_vector_json'])
print(type(first_vector))        # should be <class 'list'>
print(len(first_vector))         # should be 768
print(type(first_vector[0]))     # should be float


Index(['Message ID', 'embedding_vector_json'], dtype='object')
Message ID                                                               0
embedding_vector_json    [-0.318, -0.183, -0.222, -0.516, -0.126, -0.35...
Name: 0, dtype: object
<class 'list'>
768
<class 'float'>


Submit "embeddings.tsv.gz" in Gradescope.

## Part 4: Train a Linear Regression

Train an ordinary least squares regression for spam/ham status where spam is treated as target value 1 and ham is treated as target value 0 with your embeddings above as the only input variables.


In [18]:
# YOUR CHANGES HERE

# Load embeddings file
df_emb = pd.read_csv("embeddings.tsv.gz", sep="\t")

# Load original dataset (already loaded earlier)
df_orig = enron_spam_data.copy()

# Fix column name to match
df_orig.rename(columns={"Spam/Ham": "label"}, inplace=True)

# Convert labels: spam → 1, ham → 0
df_orig["label_num"] = df_orig["label"].map({"spam": 1, "ham": 0})

# Merge on row index (both have same order: 0..N-1)
df = pd.concat([df_orig, df_emb], axis=1)


In [19]:
# Parse JSON embeddings into list of floats
X = np.vstack(df["embedding_vector_json"].apply(json.loads).values)

y = df["label_num"].values


In [24]:
print(X.shape)
print(y.shape)

(33716, 768)
(33716,)


In [21]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X, y)


LinearRegression()

Save the coefficients of your linear model in a file "coefficients.tsv" with columns dim and coefficient where dim specifies the offset in the embedding vector (0-767).
Don't worry about the bias term (but your model should still have it).

In [22]:
# YOUR CHANGES HERE

coef_df = pd.DataFrame({
    "dim": np.arange(768),
    "coefficient": model.coef_
})

coef_df.to_csv("coefficients.tsv", sep="\t", index=False)
print("Saved coefficients.tsv")


Saved coefficients.tsv


In [23]:
print(coef_df.shape)
coef_df.head()


(768, 2)


,dim,coefficient
0,0,0.153022
1,1,0.182857
2,2,0.219427
3,3,0.166761
4,4,0.199420


Submit "coefficients.tsv" in Gradescope.

In [26]:
import pandas as pd, numpy as np, json
from pathlib import Path

# 1) file exists and columns are correct
print("coefficients.tsv exists:", Path("coefficients.tsv").exists())
coef_check = pd.read_csv("coefficients.tsv", sep="\t")
print("columns:", coef_check.columns.tolist())
print("rows:", coef_check.shape)

# 2) correct dim range and unique dims
print("dim min/max:", coef_check["dim"].min(), coef_check["dim"].max())
print("any missing dims?", set(range(768)) - set(coef_check["dim"].astype(int).tolist()))

# 3) coefficient dtype
print("coefficient dtype:", coef_check["coefficient"].dtype)

# 4) verify model coef length against expected embedding dim
print("len(model.coef_):", len(model.coef_))
assert len(model.coef_) == 768, "Coefficient length is not 768!"

# 5) check labels mapping (spam -> 1, ham -> 0) and no NaNs in y
print("label values:", np.unique(y))
print("any NaNs in X or y?", np.isnan(X).any(), np.isnan(y).any())

# 6) validate embeddings shape loaded from file (sanity)
df_emb = pd.read_csv("embeddings.tsv.gz", sep="\t")
first_vec = json.loads(df_emb.iloc[0]["embedding_vector_json"])
print("first embedding length:", len(first_vec))

coefficients.tsv exists: True
columns: ['dim', 'coefficient']
rows: (768, 2)
dim min/max: 0 767
any missing dims? set()
coefficient dtype: float64
len(model.coef_): 768
label values: [0 1]
any NaNs in X or y? False False
first embedding length: 768


In [27]:
google.colab.files.download("coefficients.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Part 5: Search for Relevant Documents

The file "queries.tsv" specifies 10 queries.
For each of the queries, encode them as a vector, and find the message that is closest using $L_2$.

In [29]:
# Part 5 computation cell — DO NOT SAVE FILE HERE

import json
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from pathlib import Path
from tqdm.auto import tqdm

# ---------- Config ----------
EMBEDDINGS_FILE = "embeddings.tsv.gz"
QUERIES_FILE = "queries.tsv"
MODEL_NAME = "bert-base-uncased"
MAX_LEN = 512
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ---------- Load email embeddings ----------
df_emb = pd.read_csv(EMBEDDINGS_FILE, sep="\t")
message_ids = df_emb["Message ID"].astype(str).tolist()
emb_matrix = np.vstack(df_emb["embedding_vector_json"].apply(json.loads).tolist())
n_messages, D = emb_matrix.shape

# ---------- Load queries ----------
df_q = pd.read_csv(QUERIES_FILE, sep="\t", dtype=str)
queries = df_q.rename(columns={df_q.columns[0]: "query_id", df_q.columns[1]: "query_text"})

# ---------- Load model ----------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)
model.to(device).eval()

@torch.no_grad()
def embed_text_batch(texts, max_len=MAX_LEN):
    enc = tokenizer(texts, padding=True, truncation=True, max_length=max_len, return_tensors="pt")
    outputs = model(**{k: v.to(device) for k, v in enc.items()})
    return outputs.last_hidden_state[:, 0, :].cpu().numpy()

# ---------- Encode queries ----------
query_matrix = np.vstack([
    embed_text_batch([t])[0]
    for t in tqdm(queries["query_text"].tolist(), desc="Encoding queries")
])

# ---------- Vectorized L2 search ----------
emb_sq = (emb_matrix**2).sum(axis=1)
qry_sq = (query_matrix**2).sum(axis=1)
dots = query_matrix @ emb_matrix.T
dists2 = qry_sq[:,None] + emb_sq - 2*dots
best_idxs = np.argmin(dists2, axis=1)

# ---------- Build output DataFrame ----------
rows = []
for i, qid in enumerate(queries["query_id"]):
    rows.append({
        "query_id": qid,
        "query_vector_json": json.dumps(query_matrix[i].tolist()),
        "Message ID": message_ids[best_idxs[i]]
    })

out_df = pd.DataFrame(rows)
out_df.head()


Using device: cuda


Encoding queries:   0%|          | 0/10 [00:00<?, ?it/s]

,query_id,query_vector_json,Message ID
0,1,"[-0.06660816073417664, 0.4860211908817291, -0....",2542
1,2,"[-0.8651902675628662, -0.24639812111854553, 0....",2758
2,3,"[-0.29763638973236084, -0.20275259017944336, -...",31624
3,4,"[-0.21511666476726532, 0.19881443679332733, -0...",13222
4,5,"[-0.19621704518795013, -0.0485721193253994, -0...",20399


Save your results in a file "query-matches.tsv" with columns query_id, query_vector_json, and Message ID.

In [30]:
# YOUR CHANGES HERE

# Saving cell — REQUIRED OUTPUT
out_df.to_csv("query-matches.tsv", sep="\t", index=False)
print("Saved query-matches.tsv")


Saved query-matches.tsv


In [31]:
print(out_df.shape)
out_df.head()

(10, 3)


,query_id,query_vector_json,Message ID
0,1,"[-0.06660816073417664, 0.4860211908817291, -0....",2542
1,2,"[-0.8651902675628662, -0.24639812111854553, 0....",2758
2,3,"[-0.29763638973236084, -0.20275259017944336, -...",31624
3,4,"[-0.21511666476726532, 0.19881443679332733, -0...",13222
4,5,"[-0.19621704518795013, -0.0485721193253994, -0...",20399


Submit "query-matches.tsv" in Gradescope.

In [33]:
google.colab.files.download("query-matches.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Part 6: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 7: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.